In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 248 kB 8.9 MB/s 
     |████████████████████████████████| 311 kB 56.1 MB/s 
     |████████████████████████████████| 1.2 MB 56.5 MB/s 
     |████████████████████████████████| 6.8 MB 48.0 MB/s 
     |████████████████████████████████| 3.5 MB 62.4 MB/s 
     |████████████████████████████████| 1.7 MB 61.6 MB/s 
     |████████████████████████████████| 43 kB 2.8 MB/s 
     |████████████████████████████████| 9.7 MB 49.5 MB/s 
     |████████████████████████████████| 67 kB 6.8 MB/s 
     |████████████████████████████████| 895 kB 68.6 MB/s 
     |████████████████████████████████| 596 kB 64.5 MB/s 
     |████████████████████████████████| 143 kB 74.2 MB/s 
     |████████████████████████████████| 180 kB 71.3 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 133 kB 71.8 MB/s 
     |████████████████████████████████| 243 kB 67.9 MB/s 
     |████████████████████████████████| 1.1 MB 58.1 MB/s 
     |███████████████

In [ ]:
!pip install wandb

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import string
from tqdm import tqdm_notebook as tqdm

from gensim.models import *

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import *
from keras.models import *
import keras.backend as K
from keras.utils import np_utils

from keras.callbacks import *

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from keras import regularizers
from numpy import array
from numpy import asarray
from numpy import zeros

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import nltk
from nltk.corpus import stopwords



In [ ]:
nltk.download('stopwords')
Stopwords = stopwords.words("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
import pandas as pd
data_train=pd.read_csv("/content/drive/MyDrive/train.tsv",sep='\t')
data_dev=pd.read_csv("/content/drive/MyDrive/dev_with_labels.tsv",sep='\t')

data_dev.head()

,PID,Text_data,Label
0,dev_pid_1,"I enjoyed today, and I still am! Tomorrows dep...",moderate
1,dev_pid_2,I sorta tried to kill myself : I had a total b...,moderate
2,dev_pid_3,Best suicide method? : I like it quick and eas...,moderate
3,dev_pid_4,a story : I remember the time I'd get on my 3D...,moderate
4,dev_pid_5,The world only cares about beautiful people : ...,moderate


In [ ]:
import string
import re
def clean_text(text):
    #text = remove_emoji(text)
    #text = re.sub('@[a-zA-Z0-9]*', text)
    #stop_words=open(r'/content/drive/MyDrive/hindiST.txt', encoding='utf=8').read().split('\n')
    text = "".join([word.lower() for word in text if word not in string.punctuation]) 
    text = "".join([word for word in text if not word.isdigit()])
    #tokens = re.split('\W+',text)
    text = " ".join(word for word in text.split() if word not in Stopwords)
    return text

data_train['clean'] = data_train['Text_data'].apply(lambda x:clean_text(x))      
data_train.head(10)

data_dev['clean'] = data_dev['Text_data'].apply(lambda x:clean_text(x))      
data_dev.head(10)

#data_test['clean'] = data_test['Text'].apply(lambda x:clean_text(x))      
#data_test.head()

,PID,Text_data,Label,clean
0,dev_pid_1,"I enjoyed today, and I still am! Tomorrows dep...",moderate,enjoyed today still tomorrows depression wait ...
1,dev_pid_2,I sorta tried to kill myself : I had a total b...,moderate,sorta tried kill total breakdown fucking car p...
2,dev_pid_3,Best suicide method? : I like it quick and eas...,moderate,best suicide method like quick easy deformitie...
3,dev_pid_4,a story : I remember the time I'd get on my 3D...,moderate,story remember time id get ds play nintendogs ...
4,dev_pid_5,The world only cares about beautiful people : ...,moderate,world cares beautiful people im born ugly ive ...
5,dev_pid_6,I try to be social. Guess what fucking happens...,moderate,try social guess fucking happens one wants tal...
6,dev_pid_7,I don't deserve love or affection. : I honestl...,moderate,dont deserve love affection honestly starting ...
7,dev_pid_8,"What am i? : I'm super depressed i guess, i do...",moderate,im super depressed guess dont want go dont wan...
8,dev_pid_9,Help : I don’t know what to do. I cant survive...,moderate,help don’t know cant survive rest school year ...
9,dev_pid_10,things aren’t getting any better : i’ve done w...,moderate,things aren’t getting better i’ve done people ...


### ***Implementation starts here***

In [ ]:
len(data_train["Label"]),len(data_train['Text_data']),len(data_dev['Text_data']),len(data_dev['Label'])

(8891, 8891, 4496, 4496)

In [ ]:
data_train["Label"].value_counts()

moderate          6019
not depression    1971
severe             901
Name: Label, dtype: int64

In [ ]:
list_of_class={'moderate':0,'not depression':1,"severe":2}
y_train=[list_of_class[i.strip()]for i in data_train["Label"]]
y_dev=[list_of_class[i.strip()]for i in data_dev["Label"]]

In [ ]:
X_train=data_train["clean"].tolist()
X_dev= data_dev["clean"].tolist()

In [ ]:
X_train[0]

'waiting mind breakdown “new year” feeling isn’t anymore don’t know anyone else i’m little bit worried i’ll go back depressed days time something last year tried breakdowns start mere days later broke crying wasn’t entire year december ok month wait it’s weird way act feel feels bit normal'

In [ ]:
X_dev[0]

'enjoyed today still tomorrows depression wait today im warm cozy even tea complement sandwich crazy time live beginning world going shambles alone feels werid stranger came saved course debt ill repay im grateful debt crazy dollars change someones situation days dont ability relax im full clean warm tomorrow back reality motel get ourself together panicking past two weeks last days exspecially got blessed week thats sure two weeks till payday well cant pull another loan gotta make whats left last planet fitness anyone okay actually gas wise idk finding safe places park extended periods time werid youd surprised much stand everyone panicking staying home night yet also kinda like motel would bucks two weeks maybe scrap car room know remember back still savings like big deal dont get wrong aint nothing sneeze back like june im seriously considering scraping car guarantee place stay always save months get another one idk today im actually less sad guess first time like two weeks could re

In [ ]:
y_test[0]

1

In [ ]:
df_train=pd.DataFrame(X_train,y_train)
df_dev=pd.DataFrame(X_dev,y_dev)

In [ ]:
df_dev.head()

,0
0,enjoyed today still tomorrows depression wait ...
0,sorta tried kill total breakdown fucking car p...
0,best suicide method like quick easy deformitie...
0,story remember time id get ds play nintendogs ...
0,world cares beautiful people im born ugly ive ...


In [ ]:
df_train.head()

,0
0,waiting mind breakdown “new year” feeling isn’...
0,new years resolution im gonna get ass therapis...
0,new year somone else feeling like last year ea...
0,story guess hi im germany english mostly self ...
0,sat dark cried going new year great start


In [ ]:
df_train.to_csv('sign_dep_train.csv',sep='\t',header=False)
df_dev.to_csv('sign_dep_dev.csv',sep='\t',header=False)

In [ ]:
df_train=pd.read_csv('sign_dep_train.csv',sep='\t',names=['label','text'])
df_dev=pd.read_csv('sign_dep_dev.csv',sep='\t',header=None)


In [ ]:
df_train=df_train[['text','label']]


In [ ]:
df_dev.columns = ["label","text"]
#df_train.columns = ["label","text"]

In [ ]:
df_dev.head()

,label,text
0,0,enjoyed today still tomorrows depression wait ...
1,0,sorta tried kill total breakdown fucking car p...
2,0,best suicide method like quick easy deformitie...
3,0,story remember time id get ds play nintendogs ...
4,0,world cares beautiful people im born ugly ive ...


In [ ]:
df_train['label'].value_counts()

0    6019
1    1971
2     901
Name: label, dtype: int64

In [ ]:
df_dev['label'].value_counts()

0    2306
1    1830
2     360
Name: label, dtype: int64

In [ ]:
model = ClassificationModel('bert', 'bert-base-uncased', num_labels=3, args={'reprocess_input_data': True, 'overwrite_output_dir': True}) 

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
model.train_model(df_train)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/8891 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1112 [00:00<?, ?it/s]

(1112, 0.5521378196983618)

In [ ]:
#result, model_outputs, wrong_predictions = model.eval_model(df_test)

In [ ]:
list_text = []
for text in df_dev['text']:
  list_text.append(text) 


In [ ]:
predictions, raw_outputs = model.predict(list_text)

  0%|          | 0/4496 [00:00<?, ?it/s]

  0%|          | 0/562 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_dev, predictions, target_names=list_of_class))

                precision    recall  f1-score   support

      moderate       0.60      0.76      0.67      2306
not depression       0.69      0.36      0.48      1830
        severe       0.31      0.50      0.38       360

      accuracy                           0.58      4496
     macro avg       0.53      0.54      0.51      4496
  weighted avg       0.61      0.58      0.57      4496



In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

label = le.fit(data_dev["Label"])
labe=le.fit_transform(data_dev["Label"])
final_prediction=le.inverse_transform(predictions)

In [ ]:
final_prediction

array(['moderate', 'severe', 'not depression', ..., 'severe', 'moderate',
       'severe'], dtype=object)